In [15]:
import requests

# Sending request to web server
payload = { 'plaintext' : 'abcdef0123456789' }
response = requests.post('http://3.90.169.27:8000/crypto4/', data=payload)

ciphertext = response.text

# Attacking Block 1

saved = 



# Sending request to web server
payload = { 'ciphertext' : ciphertext }
response = requests.post('http://3.90.169.27:8000/crypto4/', data=payload)

print(response.text)

OK
